<a href="https://colab.research.google.com/github/Shashwat17-vit/Comparative-Study-of-FaceAge-Editing-Using-Diffusion-Models/blob/main/Diffusion%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install torch diffusers transformers pillow requests

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write

In [2]:
!pip install torch torchvision transformers diffusers huggingface-hub

In [6]:
import torch
from diffusers import DDPMPipeline
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
import numpy as np

# 1. Load CLIP model and processor from Hugging Face
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# 2. Load the DDPM model from Hugging Face's diffusers library
pipe = DDPMPipeline.from_pretrained("google/ddpm-ema-bedroom-256", use_auth_token=True)
pipe.to(device)

# Function to load and preprocess the input image (face)
def load_image(image_path):
    if image_path.startswith('http'):
        response = requests.get(image_path)
        img = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        img = Image.open(image_path).convert("RGB")
    return img

# Function to update the generated image using the CLIP similarity score
def update_image_with_clip(image, target_age_prompt):
    # Preprocess text prompt and image for CLIP model
    text_input = clip_processor(text=[target_age_prompt], return_tensors="pt", padding=True).to(device)
    image_input = clip_processor(images=image, return_tensors="pt").to(device)

    # Get embeddings from CLIP model
    image_embedding = clip_model.get_image_features(**image_input)
    text_embedding = clip_model.get_text_features(**text_input)

    # Calculate cosine similarity between image and text
    similarity_score = torch.cosine_similarity(image_embedding, text_embedding)
    return similarity_score.item()

# Function to generate the aged version of the image using the DDPM model
def generate_aged_face(original_face_image, target_age_prompt, num_iterations=10, similarity_threshold=0.85):
    # Convert input image to tensor and normalize (range [0, 1])
    original_face_image = original_face_image.convert("RGB")
    original_face_image = original_face_image.resize((256, 256))  # Resize to 256x256 for DDPM model

    init_image = np.array(original_face_image) / 255.0  # Normalize image
    init_image = torch.tensor(init_image).permute(2, 0, 1).unsqueeze(0).to(device).float()  # (C, H, W) format

    # Generate the initial face image with DDPM model (using random noise)
    noisy_image = torch.randn_like(init_image)  # Start with random noise
    generated_image = pipe(noisy_image).images[0]

    # Start the feedback loop to adjust the image
    for iteration in range(num_iterations):
        print(f"Iteration {iteration + 1}/{num_iterations}")

        # Evaluate the similarity between the generated image and the target age using CLIP
        similarity_score = update_image_with_clip(generated_image, target_age_prompt)
        print(f"Similarity Score: {similarity_score:.4f}")

        if similarity_score >= similarity_threshold:
            print("Target age reached!")
            break

        # Regenerate the image using the DDPM model (refining the noisy image)
        noisy_image = torch.randn_like(init_image)  # Adjust noise as needed
        generated_image = pipe(noisy_image).images[0]

    return generated_image

# Main execution
if __name__ == "__main__":
    # Load the input face image (path or URL)
    input_image_path = "image1.jpg"  # Replace with your image path or URL
    original_face_image = load_image(input_image_path)

    # Specify the target age description (e.g., "40-year-old man")
    target_age_prompt = "A 40-year-old man"  # Example target age

    # Generate and update the face to match the desired age
    final_image = generate_aged_face(original_face_image, target_age_prompt)

    # Save or display the final result
    final_image.save("aged_face_output.jpg")
    final_image.show()


Keyword arguments {'use_auth_token': True} are not expected by DDPMPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--google--ddpm-ema-bedroom-256/snapshots/df329a8d5319a212e813ba83427e4af56760c35f: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--google--ddpm-ema-bedroom-256/snapshots/df329a8d5319a212e813ba83427e4af56760c35f.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


TypeError: randn() received an invalid combination of arguments - got (tuple, layout=torch.layout, dtype=torch.dtype, device=torch.device, generator=NoneType), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, torch.Generator generator, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)


To learn more about accelerating pandas on Colab, see the [10 minute guide](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_colab_demo.ipynb) or
 [US stock market data analysis demo](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_stocks_demo.ipynb).